In [ ]:
!pip install tensorflow
!pip install jax
!pip install tensorcircuit-ng
!pip install optax

In [2]:
import tensorcircuit as tc
import tensorflow as tf

K = tc.set_backend("tensorflow")

n = 1


def loss(params, n):
    c = tc.Circuit(n)
    for i in range(n):
        c.rx(i, theta=params[0, i])
    for i in range(n):
        c.rz(i, theta=params[1, i])
    loss = 0.0
    for i in range(n):
        loss += c.expectation([tc.gates.z(), [i]])
    return tf.math.real(loss)

def vgf(params, n):
    with tf.GradientTape() as tape:
        tape.watch(params)
        l = loss(params, n)
    return l, tape.gradient(l, params)

vgf = tf.function(vgf)
params = tf.random.normal([2, n])
print(vgf(params, n))  # get the quantum loss and the gradient

(<tf.Tensor: shape=(), dtype=float32, numpy=0.7505349516868591>, <tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[-6.6083074e-01],
       [-1.4901161e-08]], dtype=float32)>)


In [3]:
import tensorcircuit as tc

K = tc.set_backend("tensorflow")

n = 1


def loss(params, n):
    c = tc.Circuit(n)
    for i in range(n):
        c.rx(i, theta=params[0, i])
    for i in range(n):
        c.rz(i, theta=params[1, i])
    loss = 0.0
    for i in range(n):
        loss += c.expectation([tc.gates.z(), [i]])
    return K.real(loss)


vgf = K.jit(K.value_and_grad(loss), static_argnums=1)
params = K.implicit_randn([2, n])
print(vgf(params, n))  # get the quantum loss and the gradient

(<tf.Tensor: shape=(), dtype=float32, numpy=0.8647469282150269>, <tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[-0.5022077],
       [ 0.       ]], dtype=float32)>)
